In [ ]:
%pip install pandas numpy matplotlib seaborn tqdm gensim pyldavis pymupdf

In [ ]:
import os
import glob
import re
import collections
import itertools

import fitz  # PyMuPDF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from wordcloud import WordCloud

In [ ]:

plt.style.use('seaborn-v0_8-darkgrid')
tqdm.pandas()


In [ ]:
# 1. Чтение PDF и извлечение текста
pdf_dir = '../data'

pdf_files = glob.glob(os.path.join(pdf_dir, '**/*.pdf'), recursive=True)
print(f'Найдено PDF файлов: {len(pdf_files)}')

def extract_text(path):
    try:
        doc = fitz.open(path)
        return '\n'.join(page.get_text('text') for page in doc)
    except Exception as e:
        print(f'Ошибка чтения {path}: {e}')
        return ''

texts = [extract_text(f) for f in tqdm(pdf_files, desc='Извлекаем текст из PDF')]
df = pd.DataFrame({'file': pdf_files, 'text': texts})
df['char_len'] = df['text'].str.len()
df['word_len'] = df['text'].str.split().str.len()
print(df[['file', 'char_len', 'word_len']].head())

In [ ]:
# 2. Очистка и предварительная обработка текста
stop_en = set([
    'and', 'the', 'for', 'with', 'that', 'this', 'from', 'are', 'was', 'not',
    'but', 'all', 'table', 'figure', 'fig', 'mm', 'wt', 'pct'
])
stop_ru = set([
    'и', 'в', 'во', 'на', 'с', 'к', 'за', 'от', 'по', 'как', 'но', 'то', 'же',
    'для', 'рис', 'табл', 'мм', 'г', 'мкм'
])
other_noise = {'угс', 'jni', 'ppm'}

stopwords = stop_en | stop_ru | other_noise

def tokenize(text):
    # Нижний регистр, слова длиной 3+ буквы на латинице или кириллице
    tokens = re.findall(r'[a-zа-яё\-]{3,}', text.lower())
    return [t for t in tokens if t not in stopwords]

df['tokens'] = df['text'].progress_apply(tokenize)
print(df[['file', 'tokens']].head())
all_tokens = list(itertools.chain.from_iterable(df['tokens']))
tok_freq = collections.Counter(all_tokens)
print(f"Всего уникальных токенов: {len(tok_freq)}")

In [ ]:
# 3. Частотный анализ
top20 = tok_freq.most_common(20)
print('Топ-20 слов по частотам:')
for word, count in top20:
    print(f'{word}: {count}')

top20_df = pd.DataFrame(top20, columns=['token', 'freq'])
# Удаляем строки с некорректными частотами
top20_df['freq'] = pd.to_numeric(top20_df['freq'], errors='coerce')
top20_df = top20_df.dropna(subset=['freq'])

if top20_df.empty:
    print('Нет данных для построения топ-20 токенов (частотный словарь пуст или данные некорректны).')
else:
    top20_df.plot.bar(
        x='token', y='freq', figsize=(12, 5), title='Топ-20 слов',
        legend=False
    )
    plt.xlabel('Термин')
    plt.ylabel('Частота')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 4. Облако слов
if tok_freq:
    wc = WordCloud(
        width=900, height=700,
        background_color='white',
        colormap='magma'
    ).generate_from_frequencies(tok_freq)

    plt.figure(figsize=(10, 7))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title('Облако слов корпуса металлургии')
    plt.show()
else:
    print("Облако слов не построено: словарь частот пуст.")

In [ ]:
# 5. Распределение длины файлов
if not df.empty:
    plt.figure(figsize=(9, 4))
    sns.histplot(df['char_len'], bins=30, kde=True)
    plt.title('Распределение длины текстов (символы)')
    plt.xlabel('Число символов')
    plt.ylabel('Количество PDF')
    plt.show()

    plt.figure(figsize=(9, 4))
    sns.histplot(df['word_len'], bins=30, kde=True)
    plt.title('Распределение длины текстов (слова)')
    plt.xlabel('Число слов')
    plt.ylabel('Количество PDF')
    plt.show()
else:
    print("Нет данных о длине файлов — DataFrame пуст.")

In [ ]:
# 6. Доменные термины (пример)
domain_terms = [
    'copper', 'slag', 'alloy', 'smelting', 'molten', 'metal', 'casting', 'flux',
    'blast furnace', 'rolling', 'coil', 'foundry', 'converter', 'sinter', 'refining',
    'ore', 'steel', 'temperature', 'oxidation', 'impurity', 'electrolysis'
]

domain_counts = {term: tok_freq.get(term, 0) for term in domain_terms}
print('Частоты доменных терминов в корпусе:')
for term, count in domain_counts.items():
    print(f'{term}: {count}')

In [ ]:
# 7. Сохранение результата
df.to_csv('pdf_metallurgy_corpus_simple.csv', index=False)
print('Результаты анализа сохранены в: pdf_metallurgy_corpus_simple.csv')